In [1]:
'''This example demonstrates the use of Convolution1D for text classification.

Gets to 0.89 test accuracy after 2 epochs.
90s/epoch on Intel i5 2.4Ghz CPU.
10s/epoch on Tesla K40 GPU.

'''

'This example demonstrates the use of Convolution1D for text classification.\n\nGets to 0.89 test accuracy after 2 epochs.\n90s/epoch on Intel i5 2.4Ghz CPU.\n10s/epoch on Tesla K40 GPU.\n\n'

In [2]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

Using TensorFlow backend.


In [3]:
# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

In [4]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [5]:
x_train

array([ list([1, 382, 24, 61, 512, 21, 114, 16, 527, 17, 16, 116, 34, 2406, 1602, 5, 4220, 2, 1148, 2102, 82, 468, 3148, 11, 217, 7, 112, 2, 133, 137, 23, 2, 138, 122, 14, 359, 8, 30, 1926, 14, 2, 114, 2, 82, 2, 2, 62, 28, 77, 128, 69, 59, 69, 53, 1161, 18, 6, 649, 62, 127, 20, 353, 8, 135, 24, 76]),
       list([1, 1976, 5, 2, 16, 1960, 691, 12, 16, 6, 1677, 125, 250, 310, 7, 995, 5, 2, 21, 329, 163, 5, 36, 343, 99, 76, 978, 2, 5, 228, 96, 99, 76, 978, 2, 5, 228, 48, 25, 942, 72, 12, 517, 46, 4233, 357, 5, 484, 4485, 12, 152, 1261, 56, 1062, 1764, 515, 5, 13, 16, 547, 53, 3571, 17, 14, 1835, 2937, 23, 279, 4, 64, 155, 15, 1894, 72, 39, 743, 14, 20, 6, 300, 16, 4, 236, 1088, 234, 13, 258, 12, 643, 441, 5, 221, 17, 12, 2, 4, 130, 21, 15, 16, 4, 64, 173, 82, 13, 426, 339, 21, 40, 4923, 2548, 5, 2, 2, 105, 6, 117, 60, 151, 14, 20, 161, 79, 101, 919, 39, 72, 12, 828, 61, 692, 13, 586, 135, 8, 340, 798, 14, 20, 21, 50, 26, 3086, 7, 128, 108, 18, 25, 8, 1142, 129, 58, 5, 278, 23, 74, 1976, 5

In [6]:
y_train

array([0, 0, 0, ..., 1, 0, 1])

In [7]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)


In [8]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

Build model...


In [9]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 50)           250000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 398, 250)          37750     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_2 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 250)               0         
__________

In [10]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 260s 10ms/step - loss: 0.4159 - acc: 0.7885 - val_loss: 0.2904 - val_acc: 0.8746
Epoch 2/2
25000/25000 [==============================] - 263s 11ms/step - loss: 0.2313 - acc: 0.9066 - val_loss: 0.2595 - val_acc: 0.8918


In [11]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 27s 1ms/step
Test score: 0.259518016238
Test accuracy: 0.8918
